In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.metrics import MeanSquaredError
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras.layers.experimental import preprocessing
from collections import Counter

def build_model(features, norm, regularizer=None, n = 0, eta = 0.001,):
  """
  Returns the build function for a neural network with n layers
  
  Keyword Arguments:
  features - the numer of features, used to define the input dimension for layers
  norm - normalization layer that can be added to normalize the data
  n - the number of hidden layers to use
  eta - learning rate for the optimizer
  """
  model = Sequential()

  if norm is not None:
    model.add(norm)

  if n == 2:
    model.add(Dense(2 * features, input_dim = features, kernel_initializer = "normal", activation='relu', kernel_regularizer=regularizer))

  if n >= 1:
    model.add(Dense(features, kernel_initializer = "normal", activation='relu', kernel_regularizer=regularizer))

  model.add(Dense(1, kernel_initializer = "normal", kernel_regularizer=regularizer))

  opt = keras.optimizers.Adam(learning_rate=eta)
  model.compile(loss='mean_squared_error', optimizer=opt, metrics=[])

  return model


def rSq(x, y, layers, reg):
  """Calculates the r squared of the given model
  
  x - the features in the data
  y - target data
  layers - neural net hidden layer number
  reg - the regularizer to use, l1 or l2
  """
  normalizer = preprocessing.Normalization(input_shape=[x.shape[1],])
  normalizer.adapt(x)
  model = build_model(x.shape[1], None, reg, layers)
  model.fit(x, y, epochs = 100, batch_size = 5, verbose = 0)

  y_hat = model.predict(x)
  mse = MeanSquaredError()
  sst = mse(y, np.mean(y)).numpy()
  mse = MeanSquaredError()
  sse = mse(y, y_hat).numpy()
  return (1 - (sse/sst))

def rSq_adj(x, y, layers, reg):
  """Calculates the adjusted r squared of the given model
  
  x - the features in the data
  y - target data
  layers - neural net hidden layer number
  reg - the regularizer to use, l1 or l2"""
  normalizer = preprocessing.Normalization(input_shape=[x.shape[1],])
  normalizer.adapt(x)
  model = build_model(x.shape[1], None, reg, layers)
  model.fit(x, y, epochs = 100, batch_size = 5, verbose = 0)
  rsq = rSq(x, y, layers, reg)
  top = (1 - rsq) * (len(y) - 1)
  bottom = (len(y) - model.count_params() - 1)
  return (1 - (top/bottom))

def rSq_cv(x, y, layers, reg):
  """Calculates the cross validated r squared of the given model
  
  x - the features in the data
  y - target data
  layers - neural net hidden layer number
  reg - the regularizer to use, l1 or l2"""
  regressor = KerasRegressor(build_fn = build_model, features=x.shape[1], norm=None, regularizer=None, n=layers, epochs = 100, batch_size = 5, verbose = 0)
  kfold = KFold(n_splits = 10)
  mse = MeanSquaredError()
  sst = mse(y, np.mean(y)).numpy()
  sse = np.mean(np.abs(cross_val_score(regressor, x, y, cv = kfold)))
  return (1 - (sse/sst))

def AIC(x, y, layers, reg):
  """Computes and returns the aic for a trained model.
  
  x - the features in the data
  y - target data
  layers - neural net hidden layer number
  reg - the regularizer to use, l1 or l2"""
  normalizer = preprocessing.Normalization(input_shape=[x.shape[1],])
  normalizer.adapt(x)
  model = build_model(x.shape[1], None, reg, layers)
  model.fit(x, y, epochs = 100, batch_size = 5, verbose = 0)
  y_hat = model.predict(x)
  mse = MeanSquaredError()
  sse = mse(y, y_hat).numpy()
  aic = len(y)*np.log(sse) + 2*model.count_params()
  return aic


def metric(x, y, layers, reg, criteria):
  """Returns a metric value for a specific Neural Net based on layers and given criteria
  
  x - the features in the data
  y - target data
  layers - neural net hidden layer number
  reg - the regularizer to use, l1 or l2
  criteria - a metric to measure for a model, options are rsq, rsq_adj, rsq_cv, aic
  """
  if criteria == "rsq":
    return rSq(x, y, layers, reg)
  elif criteria == "rsq_adj":
    return rSq_adj(x, y, layers, reg)
  elif criteria == "rsq_cv":
    return rSq_cv(x, y, layers, reg)
  elif criteria == "aic":
    return AIC(x, y, layers, reg)

def forwardSel(layers, included, x, y, criteria):
  """
  Chooses the next best feature to include in the model based on the given criteria

  layers - neural net hidden layer number
  included - the features from x that have already been included in the model
  x - the features in the data
  y - target data
  criteria - the metric used to measure the model
  """
  base_x = x[included]
  best = metric(base_x.values, y.values, layers, None, criteria) # make starting model
    
  # Features yet to be included
  remaining = x.columns.tolist()
  for i in included:
    remaining.remove(i)

  new = included.copy()
        
  for i in remaining:
            
    using = included.copy()

    using.append(i)
            
    temp_x = x[using]
    new_metric = metric(temp_x.values, y.values, layers, None, criteria)
        
    # If new model with added feature is better than base, then add features to new list.
    # If metric is rsq based, then check if its greater
    if criteria.startswith('rsq'):
      if new_metric > best:
        print(using, best, new_metric)
        best = new_metric
        new = using
    # If metric is aic then check if its lesser
    elif critera == 'aic':
      if new_metric < best:
        print("aic ", best, new_metric)
        best = new_metric
        new = using

  temp_x = x[new]
  rsq = metric(temp_x.values,y.values,layers, None,'rsq')
  rsq_adj = metric(temp_x.values,y.values,layers, None,'rsq_adj')
  rsq_cv = metric(temp_x.values,y.values,layers, None,'rsq_cv')
  aic = metric(temp_x.values,y.values,layers, None, 'aic')

  return new, rsq, rsq_adj, rsq_cv, aic

def forwardSelAll(layers, data, response, metric):
  """
  Chooses the the best combination of features from the dataset using forward selection

  layers - neural net hidden layer number
  data - the entire dataset 
  response - the response variable in the dataset
  criteria - the metric used to measure the model
  """
  included = [] # The predictors used in the model
  
  # All four metrics which can be used to evaluate the model
  rsq = []
  rsq_adj = []
  rsq_cv = []
  aic = []
  num_features = []

  x = data.drop(response, axis=1)
  y = data[response]
  for i in range(x.shape[1]):

    new_features, new_rsq, new_rsq_adj, new_rsq_cv, new_aic = forwardSel(layers, included, x, y, metric)

    rsq.append(new_rsq)
    rsq_adj.append(new_rsq_adj)
    rsq_cv.append(new_rsq_cv)
    aic.append(new_aic)
    num_features.append(len(new_features))

    included = new_features

  return included, rsq, rsq_adj, rsq_cv, aic, num_features

def backwardElim(layers, included, x, y, criteria):
  """
  Removes the worst feature in the model based on the given criteria

  layers - neural net hidden layer number
  included - the features from x that are included in the model
  x - the features in the data
  y - target data
  criteria - the metric used to measure the model
  """
  base_x = x[included]
  best = metric(base_x.values, y.values, layers, None, criteria) # make starting model

  new = included.copy()
    
  if len(included) > 0:
            
    for i in included:
      using = included.copy()
      using.remove(i)
                
      temp_x = x[using] 
      new_metric = 0       
      if len(using) > 0:
        new_metric = metric(temp_x.values, y.values, layers, None, criteria)
                
      # If new model with added feature is better than base, then add features to new list.
      # If metric is rsq based, then check if its greater
      if criteria.startswith('rsq'):
        if new_metric > best:
          print(using, best, new_metric)
          best = new_metric
          new = using
      # If metric is aic then check if its lesser
      elif critera == 'aic':
        if new_metric < best:
          print("aic ", best, new_metric)
          best = new_metric
          new = using

  temp_x = x[new]
  rsq = metric(temp_x.values,y.values,layers, None,'rsq')
  rsq_adj = metric(temp_x.values,y.values,layers, None,'rsq_adj')
  rsq_cv = metric(temp_x.values,y.values,layers, None,'rsq_cv')
  aic = metric(temp_x.values,y.values,layers, None,'aic')
            
  return new, rsq, rsq_adj, rsq_cv, aic

def backwardElimAll(layers, data, response, metric):
  """
  Chooses the the best combination of features from the dataset using backward elimination

  layers - neural net hidden layer number
  data - the entire dataset 
  response - the response variable in the dataset
  criteria - the metric used to measure the model
  """
  # Start will all features
  included = data.columns.tolist()
  included.remove(response)

  rsq = []
  rsq_adj = []
  rsq_cv = []
  aic = []
  num_features = []
    
  x = data[included]
  y = data[response]
    
  for i in range(x.shape[1]):
       
    new_features, new_rsq, new_rsq_adj, new_rsq_cv, new_aic = backwardElim(layers, included, x, y, metric)

    if Counter(new_features) != Counter(included):
      rsq.append(new_rsq)
      rsq_adj.append(new_rsq_adj)
      rsq_cv.append(new_rsq_cv)
      aic.append(new_aic)
      num_features.append(len(new_features))
        
      included = new_features

  return included, rsq, rsq_adj, rsq_cv, aic, num_features

def stepwiseReg(layers, included, x, y, criteria):
  """
  Either adds the best not included feature or removes the worst included feature in the model based on the given criteria

  layers - neural net hidden layer number
  included - the features from x that are included in the model
  x - the features in the data
  y - target data
  criteria - the metric used to measure the model
  """
  base_x = x[included]
  best = metric(base_x.values, y.values, layers, None, criteria)
  rsq = metric(base_x.values, y.values, layers, None, 'rsq')
  rsq_adj = metric(base_x.values, y.values, layers, None, 'rsq_adj')
  rsq_cv = metric(base_x.values, y.values, layers, None, 'rsq_cv')
  aic = metric(base_x.values, y.values, layers, None, 'aic')
   
  final = included

  forward, new_rsq, new_rsq_adj, new_rsq_cv, new_aic = forwardSel(layers, included, x, y, criteria)
  if new_rsq_adj > best:
    print(forward, best, new_rsq_adj)
    final = forward
    best = new_rsq_adj
    rsq, rsq_adj, rsq_cv, aic = new_rsq, new_rsq_adj, new_rsq_cv, new_aic
    
  backward, new_rsq, new_rsq_adj, new_rsq_cv, new_aic = backwardElim(layers, included, x, y, criteria)
  if new_rsq_adj > best:
    print(backward, best, new_rsq_adj)
    final = backward
    best = new_rsq_adj
    rsq, rsq_adj, rsq_cv, aic = new_rsq, new_rsq_adj, new_rsq_cv, new_aic
   
  return final, rsq, rsq_adj, rsq_cv, aic

def stepwiseRegAll(layers, data, response, metric):
  """
  Chooses the the best combination of features from the dataset using stepwise regression

  layers - neural net hidden layer number
  data - the entire dataset 
  response - the response variable in the dataset
  criteria - the metric used to measure the model
  """
  included = []
  base_metric = 0

  rsq = []
  rsq_adj = []
  rsq_cv = []
  aic = []
  num_features = []

  x = data.drop(response, axis=1)
  y = data[response]
    
  while True:
        
    new_features, new_rsq, new_rsq_adj, new_rsq_cv, new_aic = stepwiseReg(layers, included, x, y, metric)
        
    rsq.append(new_rsq)
    rsq_adj.append(new_rsq_adj)
    rsq_cv.append(new_rsq_cv)
    aic.append(new_aic)
    num_features.append(len(new_features))
        
    included = new_features

    if new_rsq_adj > base_metric:
      included = new_features
      base_metric = new_rsq_adj
    else:
      break

  return included, rsq, rsq_adj, rsq_cv, aic, num_features

def Regularizer(reg, layers, data, response):
  """
  Chooses which regularizer to run the model with

  reg - the regularizer to use, l1 or l2
  layers - neural net hidden layer number
  data - the entire dataset 
  response - the response variable in the dataset
  """
  x = data.drop(response, axis=1)
  y = data[response]
  rsq = metric(x.values,y,layers,reg,'rsq')
  rsq_adj = metric(x.values,y,layers,reg,'rsq_adj')
  rsq_cv = metric(x.values,y,layers,reg,'rsq_cv')
  aic = metric(x.values,y,layers,reg,'aic')

  return rsq, rsq_adj, rsq_cv, aic


def feature_selection(layers, selection, data, response, metric):
  """
  Chooses which feature selection method to run based on selection parameter

  layers - neural net hidden layer number
  selection - the feature selection to use, options are foward, backward, stepwise
  data - the entire dataset 
  response - the response variable in the dataset
  criteria - the metric used to measure the model
  """
  if selection == 'forward':
    return forwardSelAll(layers, data, response, metric)
  elif selection == 'backward':
    return backwardElimAll(layers, data, response, metric)
  elif selection == 'stepwise':
    return stepwiseRegAll(layers, data, response, metric)

def plotModel(rsq, rsq_adj, rsq_cv, aic, num, selection):
  """
  Plots the various metrics of a model after feature selection

  rsq - the r squared values of the model after going through feature selection
  rsq_adj - the adjusted r squared values of the model after going through feature selection
  rsq_cv - the cross validated r squared values of the model after going through feature selection
  aic - the AIC values of the model after going through feature selection
  num - the number of features at each step of feature selection
  selection - the feature selection method used
  """
  plt.style.use("seaborn-bright")
  metrics = pd.DataFrame(data = {'R-Squared': rsq, 'R-Squared_Adj': rsq_adj, 'R-Squared_CV': rsq_cv}, index = num)
  plt.figure()
  r = metrics.plot(xlabel='Number of Features', ylabel='R^2', figsize=(10,8))
  if selection == 'backward':
    plt.gca().invert_xaxis()

  aic_df = pd.DataFrame(data = {'AIC': aic}, index = num)
  plt.figure()
  a = aic_df.plot(xlabel='Number of Features', ylabel='AIC', figsize=(10,8))
  if selection == 'backward':
    plt.gca().invert_xaxis()
  plt.show()
    
def runAllModels(data, response):
  """
  Runs every specified model for a dataset. Can be customized to run whatever models are needed.
  By default, runs only forward selection for a perceptron.

  data - the dataset to use
  response - target feature
  """
  rsq = []
  rsq_adj = []
  rsq_cv = []
  aic = []
  print(data.name)

  # Change list to run whichever types of neural net are wanted. 
  # [0, 1, 2] 0 for perceptron, 1 for NN3L, 2 for NNXL. Change as needed
  for layer in [0]:
    if layer == 0:
      print("<---------------Perceptron-------------->\n")
    elif layer == 1:
      print("<---------------NeuralNet3L-------------->\n")
    elif layer == 2:
      print("<---------------NeuralNetXL-------------->\n")
    
    # Change list to run whichever types of feature selection methods are wanted. 
    # ['forward', 'backward', 'stepwise']
    # forward for forward selection, backward for backward elimination, stepwise for stepwise regression

    #for method in ['forward', 'backward', 'stepwise']:
    for method in ['forward']:
      if method == 'forward':
        print("<---------------Adjusted R-Squared based Forward Selection-------------->\n")
      elif method == 'backward':
        print("<---------------Adjusted R-Squared based Backward Elimination-------------->\n")
      elif method == 'stepwise':
       print("<---------------Adjusted R-Squared based Stepwise Regression-------------->\n")


      # This is the metric used to evaluate the models in the feature selection. Any of rsq,
      # rsq_adj, rsq_cv, and aic can be used instead. The default is rsq_adj since this is what we
      # found to be best.
      metric = "rsq_adj"

      features, rsq, rsq_adj, rsq_cv, aic, num_features = feature_selection(layer, method, data, response, metric)
      print(features)
      plotModel(rsq, rsq_adj, rsq_cv, aic, num_features, method)
    
    # Change list to run either, both, or no regularization methods. 
    # ['l1', 'l2'] l1 for L1 regularization and l2 for L2 regularization
    for regularizer in []:
      rsq, rsq_adj, rsq_cv, aic = Regularizer(regularizer, layer, data, response)
      if regularizer == 'l1':
        print("<---------------L1 Regularization-------------->\n")
      elif regularizer == 'l2':
        print("<---------------L2 Regularization-------------->\n")

      print("R-Squared = " + str(rsq) + ", Adjusted R-Squared = " + str(rsq_adj) + ", R-Squared CV = " + str(rsq_cv) + ", AIC = " + str(aic) + "\n")
      






In [ ]:
url = 'https://raw.githubusercontent.com/Data-Science-II/Project2-PranayKumar-DylanFauntleroy-NathanHales/main/Datasets/auto-mpg.csv'

autompg = pd.read_csv(url)
autompg = autompg.fillna(autompg.mean())
autompg.name = 'auto-mpg'
print(autompg.dtypes)

runAllModels(autompg, 'mpg')

In [ ]:
url = 'https://raw.githubusercontent.com/Data-Science-II/Project2-PranayKumar-DylanFauntleroy-NathanHales/main/Datasets/Concrete_Data.csv'

concrete = pd.read_csv(url)
concrete = concrete.fillna(concrete.mean())
concrete.name = 'concrete'
print(concrete.dtypes)

runAllModels(concrete, 'Concrete compressive strength(MPa, megapascals) ')

In [ ]:
url = 'https://raw.githubusercontent.com/Data-Science-II/Project2-PranayKumar-DylanFauntleroy-NathanHales/main/Datasets/Red_Wine_Quality.csv'

wine = pd.read_csv(url)
wine = wine.fillna(wine.mean())
wine.name = 'wine'
print(wine.dtypes)

runAllModels(wine, 'quality')

In [ ]:
url = 'https://raw.githubusercontent.com/Data-Science-II/Project2-PranayKumar-DylanFauntleroy-NathanHales/main/Datasets/SeoulBikeData.csv'

bike = pd.read_csv(url)
bike = bike.fillna(bike.mean())
bike = bike.select_dtypes(exclude=['object'])
bike.name = 'bike'
print(bike.dtypes)

runAllModels(bike, 'Rented Bike Count')

In [ ]:
url = 'https://raw.githubusercontent.com/Data-Science-II/Project2-PranayKumar-DylanFauntleroy-NathanHales/main/Datasets/Data_for_UCI_named.csv'

electric = pd.read_csv(url)
electric = electric.fillna(electric.mean())
electric.name = 'electric'
electric = electric.drop('stabf', axis=1)
print(electric.dtypes)

runAllModels(electric, 'stab')